# PROYECTO V - PROBLEMA DE REGRESIÓN

En este proyecto de problema de regresión hemos decidido usar un [Dataset](https://www.kaggle.com/competitions/playground-series-s4e9/overview) de precio de coches usados. El proyecto está enfocado a las personas que quieran vender su coche y quieran tener una idea del precio de venta de antemano. Para poder hacer un estudio del Dataset y crear un Modelo que prediga los precios de venta del coche lo primero que vamos a hacer es una limpieza del Dataset, ver los valores faltantes o outliers y poder tratarlos.

Tras la limpieza de los datos los datos para calcular el precio del coche son:
- Características: 
    - brand
    - model
    - model_year
    - milage
    - fuel_type
    - engine
    - transmission
    - ext_col
    - int_col
    - accident
    - price
- Etiqueta: precio del coche

Vamos a ver los pasos que hemos realizado para llegar a tener las columnas listas.

## PASO 1: REVISAR EL DATASET

### Importar el dataset.

In [175]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install numpy
!pip install scikit-learn

In [218]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('data/used_cars.csv')

### Imprimir cabeceras del dataset y tener una idea clara de los tipos de datos que tenemos.

In [219]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


### Eliminar la columna 'clean_title' porque no lo vamos a usar.

In [220]:
df.drop(['clean_title'], axis=1, inplace=True)

### Imprimir los elementos y subniveles que tiene cada columna.

In [221]:
cols_cat = ['brand','model','milage','fuel_type','engine','transmission','ext_col','int_col','accident','price']

for col in cols_cat:
    print(f'Column {col}: {df[col].nunique()} subniveles')

Column brand: 57 subniveles
Column model: 1898 subniveles
Column milage: 2818 subniveles
Column fuel_type: 7 subniveles
Column engine: 1146 subniveles
Column transmission: 62 subniveles
Column ext_col: 319 subniveles
Column int_col: 156 subniveles
Column accident: 2 subniveles
Column price: 1569 subniveles


### Generar estadísticas descriptivas de las columnas numéricas de nuestro dataset.

In [222]:
df.describe()

,model_year
count,4009.000000
mean,2015.515590
std,6.104816
min,1974.000000
25%,2012.000000
50%,2017.000000
75%,2020.000000
max,2024.000000


### Mostrar todas las filas y columnas

In [223]:
# Mostrar todas las filas y columnas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

print(df)

         brand                            model  model_year      milage  \
0         Ford  Utility Police Interceptor Base        2013  51,000 mi.   
1      Hyundai                     Palisade SEL        2021  34,742 mi.   
2        Lexus                    RX 350 RX 350        2022  22,372 mi.   
3     INFINITI                 Q50 Hybrid Sport        2015  88,900 mi.   
4         Audi        Q3 45 S line Premium Plus        2021   9,835 mi.   
...        ...                              ...         ...         ...   
4004   Bentley             Continental GT Speed        2023     714 mi.   
4005      Audi             S4 3.0T Premium Plus        2022  10,900 mi.   
4006   Porsche                           Taycan        2022   2,116 mi.   
4007      Ford                     F-150 Raptor        2020  33,000 mi.   
4008       BMW                     X3 xDrive30i        2020  43,000 mi.   

          fuel_type                                             engine  \
0     E85 Flex Fuel  300.

### Limpieza de la columna 'milage'

Como vemos la columna'milage' está en tipo de dato object y además el formato no nos dejaría hacer un análisis. Por ello, hemos decidido eliminar el string 'mi.', reemplazar el ',' por '.' y cambiarlo a tipo float.

In [224]:
df['milage'] = df['milage'].str.replace('mi.', '', regex=False).str.replace(',', '.').astype(float)

### Limpieza de la columna 'price'

Hacemos lo mismo con la columna 'price', eliminando el signo de '$' y cambiando el ',' por '.'

In [225]:
df['price'] = df['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

### Limpieza de la columna 'engine'

Esta columna ha sido la más problemática, porque nos dimos cuena que había muchas diferentes opciones y variables. Para esto hemos usado expresiones regulares (regex) para buscar recuencias de carácteres y patrones comunes en esta columna y crear una nueva columna para cada uno de ellos.

In [226]:
df['engine'].unique()

array(['300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability',
       '3.8L V6 24V GDI DOHC', '3.5 Liter DOHC', ...,
       '136.0HP 1.8L 4 Cylinder Engine Gasoline Fuel',
       '270.0HP 2.0L 4 Cylinder Engine Gasoline Fuel',
       '420.0HP 5.9L 12 Cylinder Engine Gasoline Fuel'],
      shape=(1146,), dtype=object)

**Creación de la columna 'engine_size' mediante regex a partir de 'engine'** : Hemos tomado el carácter 'L' como filtro, para extraer la capacidad del motor del coche. 

Por ejemplo lo que antes era:

    ['engine] = 3.8L V6 24V GDI DOHC

Ahora es:

    ['engine_size] 3.8


In [227]:
df['engine_size'] = df['engine'].str.extract(r'(\d.\d+)\s?L').astype(float)

**Creación de la columna 'engine_type' meidante regex a partir de la columna 'engine'**: Hemos tomado como carácteres delimitantes las palabras 'I\d', 'Electric', 'Hybrid', 'Turbo', 'Diesel'.

Por ejemplo lo que antes era:

    ['engine'] = 534.0HP Electric Motor Electric Fuel System

Ahora es:

    [engine_type'] = Electric

Y además, las columnas vacías hemos reestablecido como 'Unknown'


In [228]:
df['engine_type'] = df['engine'].str.extract(r'(V\d|I\d|Electric|Hybrid|Turbo|Diesel)', expand=False)
df['engine_type'] = df['engine_type'].fillna('Unknown')

**Creación de la columna 'engine_hp' mediante regex a partir de la columna 'engine'**: Hemos tomado como carácter delimitador el string 'HP'.

Por ejemplo lo que antes era:

    ['engine'] = 534.0HP Electric Motor Electric Fuel System

Ahora es:

    [engine_hp'] = 534.0


In [229]:
df['engine_hp'] = df['engine'].str.extract(r'(\d+.?\d*)\s?HP')
df['engine_hp'] = pd.to_numeric(df['engine_hp'], errors='coerce')

### Eliminamos las columnas 'engine', 'ext_col' e 'int_color'

In [230]:
df = df.drop(columns=['engine','ext_col','int_col'])

### Normalizar la columna 'accident'

Hemos convertido en valores booleanos la columna 'accident':

    'None reported' => 0
    'At least 1 accident or damage reported' => 1
    'nan' => 0

In [231]:
df['accident'] = df['accident'].map({
    'None reported': 0,
    'At least 1 accident or damage reported': 1
}).where(pd.notna(df['accident'])).astype('Int64')

df['accident'] = df['accident'].fillna(0).astype('Int64')

### Pasando la columna price a int

In [232]:
df["price"] = df["price"].astype(int)

### Suplantando los valores vacios de Fuel_type por Electric (corroborado en la otra columna engine_type)

In [233]:
df["fuel_type"] = df["fuel_type"].fillna("Electric")

### Luego de evaluar decidimos borrar la columna engine_type

In [234]:
df.drop(['engine_type'], axis=1, inplace=True)

### Craar columnas brand_id y model_id

Para que el modelo funcione teniendo en cuenta la marca ('brand') y el modelo ('model') decidimos usar LabelEncoder para pasar de string a int.

Desde la columna 'brand' creamos una nueva columna 'brand_id' y desde la columna 'model' creamos una nueva columna 'model_id'. Además guardamos cada columna con su respectivo código en un nuevo archivo csv para tener una referencia.

In [235]:
from sklearn.preprocessing import LabelEncoder

brand_le = LabelEncoder()
model_le = LabelEncoder()

df["brand_id"] = brand_le.fit_transform(df["brand"])
df["model_id"] = model_le.fit_transform(df["model"])

# Imprimir las marcas con sus respectivos códigos
#for i, brand in enumerate(le.classes_):
    #print(f"{brand} → {i}")

import pandas as pd

# crear un archivo csv con las marcas y sus códigos
brand_mapping = {brand: i for i, brand in enumerate(brand_le.classes_)}
brand_df = pd.DataFrame(list(brand_mapping.items()), columns=["brand", "code"])
brand_df.to_csv("data/brand_enumeration.csv", index=False)

model_mapping = {model: i for i, model in enumerate(model_le.classes_)}
model_df = pd.DataFrame(list(model_mapping.items()), columns=["model", "code"])
model_df.to_csv("data/model_enumeration.csv", index=False)


print("Brands guardados en data/brand_enumeration.csv ✅")
print("Modelos guardado en data/model_enumeration.csv ✅")


Brands guardados en data/brand_enumeration.csv ✅
Modelos guardado en data/model_enumeration.csv ✅


### Reemplazar los valores '-' y 'not supported' de la columna 'fuel_type' por 'Electric'

Hemos visto que hay 45 filas que contienen '-' y 2 filas que contienen 'not supported' en la columna 'fuel_type' y damos por hecho que si pone eso es que no es ni Gasolina ni Diesel, y decidimos reemplazarlos por 'Electric'.


In [236]:
# Ver los valores únicos de la columna 'fuel_type'
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Electric', 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [237]:
# contar las filas que tienen el valor '–' en la columna 'fuel_type'
df['fuel_type'].isin(['–']).sum()

np.int64(45)

In [238]:
# contar las filas que contienen 'not supported' en la columna 'fuel_type'
df['fuel_type'].isin(['not supported']).sum()

np.int64(2)

In [239]:
# Reemplazar los valores '-' y 'not supported' por 'Electric' en la columna 'fuel_type'
df['fuel_type'] = df['fuel_type'].replace('–', 'Electric')
df['fuel_type'] = df['fuel_type'].replace('not supported', 'Electric')

In [240]:
# Mostrar los valores únicos para ver si los cambios se realizaron correctamente
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Electric', 'Diesel',
       'Plug-In Hybrid'], dtype=object)

### Crear nueva columna 'fuel_type_id' a partir de 'fuel_type' pasado a números para poder usarlos en el modelo

In [241]:
from sklearn.preprocessing import LabelEncoder

# creamos el encoder
le_fuel = LabelEncoder()

# transformamos la columna fuel_type a int
df['fuel_type_id'] = le_fuel.fit_transform(df['fuel_type'])

fuel_mapping = {fuel: i for i, fuel in enumerate(le_fuel.classes_)}
fuel_df = pd.DataFrame(list(fuel_mapping.items()), columns=["fuel_type", "code"])
fuel_df.to_csv("data/fuel_type_enumeration.csv", index=False)

print("Fuel_type guardado en data/fuel_type_enumeration.csv ✅")

Fuel_type guardado en data/fuel_type_enumeration.csv ✅


In [242]:
df['fuel_type_id'].unique()

array([1, 3, 4, 2, 0, 5])

In [243]:
df.head(5)

,brand,model,model_year,milage,fuel_type,transmission,accident,price,engine_size,engine_hp,brand_id,model_id,fuel_type_id
0,Ford,Utility Police Interceptor Base,2013,51.000,E85 Flex Fuel,6-Speed A/T,1,10300,3.7,300.0,14,1743,1
1,Hyundai,Palisade SEL,2021,34.742,Gasoline,8-Speed Automatic,1,38005,3.8,NaN,19,1182,3
2,Lexus,RX 350 RX 350,2022,22.372,Gasoline,Automatic,0,54598,3.5,NaN,27,1325,3
3,INFINITI,Q50 Hybrid Sport,2015,88.900,Hybrid,7-Speed A/T,0,15500,3.5,354.0,20,1242,4
4,Audi,Q3 45 S line Premium Plus,2021,9.835,Gasoline,8-Speed Automatic,0,34999,2.0,NaN,3,1225,3


### Limpieza en la columna 'transmission'

In [244]:
# Ver valores únicos y sus conteos de la columna 'transmission'
df['transmission'].value_counts()

transmission
A/T                                              1037
8-Speed A/T                                       406
Transmission w/Dual Shift Mode                    398
6-Speed A/T                                       362
6-Speed M/T                                       248
                                                 ... 
Automatic, 8-Spd PDK Dual-Clutch                    1
Auto, 6-Spd w/CmdShft                               1
Automatic, 8-Spd Sport w/Sport & Manual Modes       1
CVT-F                                               1
8-Speed Manual                                      1
Name: count, Length: 62, dtype: int64

In [245]:
# contar los valores únicos de 'transmission'
df['transmission'].nunique()

62

Hemos visto que hay 3 modelos de la marca 'Acura' que tienen '2' en la columna 'transmission' y tras indgar en internet y con ayuda de ChatGPT, hemos llegado a la conclusion de que dichas marcas, con ese modelo, ese tipo de combustible y de dichos años, tienen el tipo de transmisón '10-Speed A/T', por lo que procedemos a reemplazarlos.

In [246]:
# imprimimos las filas que tienen el valor '2' en 'transmission'
df[df['transmission'] == '2']

,brand,model,model_year,milage,fuel_type,transmission,accident,price,engine_size,engine_hp,brand_id,model_id,fuel_type_id
269,Acura,TLX w/A-Spec Package,2022,14.896,Gasoline,2,0,39998,2.0,NaN,0,1648,3
516,Acura,MDX w/Technology Package,2022,30.177,Gasoline,2,0,46598,3.5,NaN,0,1005,3
2381,Acura,RDX PMC Edition,2021,44.457,Gasoline,2,0,40598,2.0,NaN,0,1298,3


In [247]:
# despues de indagar, se observa que el valor '2' corresponde a '10-Speed A/T' y los reemplazamos para que no haya confusiones
df.loc[df['transmission'] == '2', 'transmission'] = '10-Speed A/T'

# comprobamos que el cambio se realizó correctamente
df[df['transmission'] == '2']

,brand,model,model_year,milage,fuel_type,transmission,accident,price,engine_size,engine_hp,brand_id,model_id,fuel_type_id


In [248]:
df['transmission'].isin(['–']).sum()

np.int64(4)

También hemos encontrado 4 filas que contienen '-' en la columna 'transmission' pero aún indagando no hemos encontrado una respuesta clara de qué tipo de transmisión tienen estos 4 vehículos, por lo que lo dejamos así para su posterior estudio.

### Expresión regular para reemplazar los valores duplicados en 'transmission'


In [249]:
df['transmission'].value_counts()

transmission
A/T                                                                 1037
8-Speed A/T                                                          406
Transmission w/Dual Shift Mode                                       398
6-Speed A/T                                                          362
6-Speed M/T                                                          248
                                                                    ... 
Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Manual Modes       1
Automatic, 8-Spd PDK Dual-Clutch                                       1
8-SPEED AT                                                             1
Auto, 6-Spd w/CmdShft                                                  1
8-Speed Manual                                                         1
Name: count, Length: 61, dtype: int64

Hay muchos valores duplicados como:

    '1-Speed Automatic' y '1-Speed A/T',

    '6-Speed Automatic' y '6-Speed A/T',
    
    Así hasta '10-Speed Automatic'

Estos pares son el mismo tipo de transmisión, por lo que decidimos reemplazar los 'n-Speed Automatic' por 'n-Speed A/T' para que haya menos uniques.

In [250]:
import re

# reemplazamos el patrón "n-Speed Automatic" por "n-Speed A/T"
df['transmission'] = df['transmission'].str.replace(r'(\d+)-Speed Automatic', r'\1-Speed A/T', regex=True)

Tambien reemplazamos otros valores que vemos duplicados:

In [251]:
# reemplazar 'Automatic' por 'A/T'
df['transmission'] = df['transmission'].str.replace('Automatic', 'A/T', regex=False)
df['transmission'] = df['transmission'].str.replace('A/T, 8-Spd', '8-Speed A/T', regex=False)
df['transmission'] = df['transmission'].str.replace('8-SPEED A/T', '8-Speed A/T', regex=False)
df['transmission'] = df['transmission'].str.replace('6-Speed Manual', '6-Speed M/T', regex=False)
df['transmission'] = df['transmission'].str.replace('6 Speed Mt', '6-Speed M/T', regex=False)
df['transmission'] = df['transmission'].str.replace('7-Speed Manual', '7-Speed M/T', regex=False)
df['transmission'] = df['transmission'].str.replace('M/T', 'Manual', regex=False)
df['transmission'] = df['transmission'].str.replace('Manual, 6-Spd', '6-Speed M/T', regex=False)
df['transmission'] = df['transmission'].str.replace('A/T, 10-Spd', '10-Speed A/T', regex=False)

In [252]:
# valores finales tras limpieza
df['transmission'].value_counts()

transmission
A/T                                                            1274
8-Speed A/T                                                     587
6-Speed A/T                                                     435
Transmission w/Dual Shift Mode                                  398
6-Speed Manual                                                  257
7-Speed A/T                                                     216
10-Speed A/T                                                    179
9-Speed A/T                                                     121
5-Speed A/T                                                      95
1-Speed A/T                                                      78
4-Speed A/T                                                      76
CVT Transmission                                                 62
5-Speed Manual                                                   59
Manual                                                           42
A/T CVT                            

In [253]:
# contar los valores únicos de 'transmission' después de la limpieza
df['transmission'].nunique()

44

### Normalizar los valores de 'transmission' para tener 3 valores únicos

0 = Es automático

1 = Manual

2 = Otros, Dual Shift

In [254]:
def normalize_transmission(value):
    if pd.isnull(value):
        return 2
    elif 'A/T' in value or 'CVT' in value:
        return 0
    elif 'M/T' in value or 'Manual' in value:
        return 1
    else:
        return 2

df['transmission_norm'] = df['transmission'].apply(normalize_transmission)


### Creamos una nueva columna 'transmission_id' a partir de la limpieza de 'transmission' convirtiendo los string a int

In [255]:
from sklearn.preprocessing import LabelEncoder

transnission_le = LabelEncoder()

df['transmission_id'] = transnission_le.fit_transform(df['transmission'])

transmission_mapping = {transmission: i for i, transmission in enumerate(transnission_le.classes_)}
transmission_df = pd.DataFrame(list(transmission_mapping.items()), columns=["transmission", "code"])
transmission_df.to_csv("data/transmission_enumeration.csv", index=False)

print("'Transmission_id' guardado en data/transmission_enumeration.csv ✅")

'Transmission_id' guardado en data/transmission_enumeration.csv ✅


In [256]:
df['engine_size'].nunique()

61

### Rellenar los campos vacíos de 'engine_hp' teniendo en cuenta el 'engine_size'

In [257]:
# Calcular la media de engine_hp para cada engine_size
hp_means_by_engine_size = df.groupby('engine_size')['engine_hp'].mean().round()

# Reemplazar valores faltantes en engine_hp usando engine_size como guía
df['engine_hp'] = df.apply(
    lambda row: hp_means_by_engine_size[row['engine_size']] 
    if pd.isna(row['engine_hp']) and not pd.isna(row['engine_size']) 
    else row['engine_hp'],
    axis=1
)

### Eliminar la columna 'engine_size'

Ahora que tenemos la columna 'engine_hp' normalizado y rellenado con los datos correspondientes, decidimos eliminar la columna 'engine_size' por ser irrelevante.

In [258]:
# eliminamos la columna 'engine_size'
df.drop('engine_size', axis=1, inplace=True)

## DATASET LIMPIO Y NORMALIZADO 👇

In [259]:
new_df = df.copy()
new_df.to_csv('data/cleaned_dataset.csv', index=False)

new_df.info()
new_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              4009 non-null   object 
 1   model              4009 non-null   object 
 2   model_year         4009 non-null   int64  
 3   milage             4009 non-null   float64
 4   fuel_type          4009 non-null   object 
 5   transmission       4009 non-null   object 
 6   accident           4009 non-null   Int64  
 7   price              4009 non-null   int64  
 8   engine_hp          3906 non-null   float64
 9   brand_id           4009 non-null   int64  
 10  model_id           4009 non-null   int64  
 11  fuel_type_id       4009 non-null   int64  
 12  transmission_norm  4009 non-null   int64  
 13  transmission_id    4009 non-null   int64  
dtypes: Int64(1), float64(2), int64(7), object(4)
memory usage: 442.5+ KB


,brand,model,model_year,milage,fuel_type,transmission,accident,price,engine_hp,brand_id,model_id,fuel_type_id,transmission_norm,transmission_id
0,Ford,Utility Police Interceptor Base,2013,51.000,E85 Flex Fuel,6-Speed A/T,1,10300,300.0,14,1743,1,0,9
1,Hyundai,Palisade SEL,2021,34.742,Gasoline,8-Speed A/T,1,38005,352.0,19,1182,3,0,20
2,Lexus,RX 350 RX 350,2022,22.372,Gasoline,A/T,0,54598,309.0,27,1325,3,0,29
3,INFINITI,Q50 Hybrid Sport,2015,88.900,Hybrid,7-Speed A/T,0,15500,354.0,20,1242,4,0,15
4,Audi,Q3 45 S line Premium Plus,2021,9.835,Gasoline,8-Speed A/T,0,34999,232.0,3,1225,3,0,20


## ANÁLISIS EXPLORATORIO DE DATOS